In [71]:
import numpy as np
from sklearn import svm
import matplotlib.pyplot as plt
import time

In [78]:
class MySVM(object):
    def __init__(self, k=None, method=None, kernel=None, max_iter=None):
        self.k = k
        self.method = method
        self.kernel = kernel
        self.iteration = max_iter

    def loadData(self, fileName):
        with open('./data/'+fileName, 'rb') as f:
            lines = f.readlines()
            y = np.zeros(len(lines), dtype=int)
            x = np.zeros((len(lines), 20), dtype=float)
            for i in range(len(lines)):
                label = int(lines[i].strip().split(' ')[0])
                y[i] = label
                feature = [float(ele.split(':')[1]) for ele in lines[i].strip().split(' ')[1:]]
                x[i] = feature
            y = y.reshape(len(lines), 1)
            return np.matrix(np.concatenate([y, x], axis=1))
        
    def svm_model(self, num_of_svm):
        modelList = []
        for i in range(num_of_svm):
            clf = svm.SVC(kernel=self.kernel, probability=True, max_iter=self.iteration)
            modelList.append(clf)
        return modelList
    
    # ----------train---------------------------------------
    def train(self):
        dataTrain = self.loadData('train.txt')
        # -------method 1: OVO------------------------------
        if self.method=='1v1':
            svmModel = []
            svmModel = self.svm_model((self.k*(self.k - 1))/2)
            dataPair = []
            for i in range(self.k-1):
                for j in range(i+1, self.k):
                    data_1 = dataTrain[np.nonzero(dataTrain[:, 0].A==i)[0]]
                    data_2 = dataTrain[np.nonzero(dataTrain[:, 0].A==j)[0]]
                    data_1[:, 0] = 1
                    data_2[:, 0] = 0
                    dataPair.append(np.vstack([data_1, data_2]))
            for k in range(len(svmModel)):
                data = dataPair[k]
                svmModel[k].fit(data[:, 1:], data[:, 0])
        # -------method 2: OVR------------------------------     
        if self.method=='1vR':
            svmModel = svmModel(self, self.k)
            dataPair = []
            for i in range(self.k):
                data_k = dataTrain[np.nonzero(dataTrain[:, 0].A==i)[0]]
                data_other = dataTrain[np.nonzero(dataTrain[:, 0].A!=i)[0]]
                data_k[:, 0] = 1
                data_other[:, 0] = 0
                dataPair.append(np.random.shuffle(np.vstack([data_k, data_other])))
            for k in len(svmModel):
                data = dataPair[k]
                svmModel[k].fit(data[:, 1:], data[:, 0]) 
        # -------method 3: PVP------------------------------       
        if self.method=='PvP':
            pass
        return svmModel
    
    # ----------test---------------------------------------
    def test(self, trained_model):
        data = self.loadData('test.txt')
        dataTest = data[:, 1:]
        predict = np.zeros([len(trained_model), data.shape[0]], dtype=float)
        for i in range(len(trained_model)):                                              
            result = trained_model[i].predict(dataTest)
            #print type(result), np.shape(result)
            predict[i] = result
        return predict
                

In [79]:
mysvm = MySVM(k=12, method='1v1', kernel='rbf', max_iter=1000)
svmModel = mysvm.train()
predict = mysvm.test(svmModel)
print predict
# print predict[1]
# print predict[2]
# print predict[3]



    

[[  0.00000000e+00   9.58610000e-02   1.08930000e-02 ...,   7.40740000e-02
    8.71500000e-03   3.26800000e-02]
 [  0.00000000e+00   1.20879000e-01   0.00000000e+00 ...,   3.84620000e-02
    1.09890000e-02   1.64840000e-02]
 [  0.00000000e+00   7.27610000e-02   7.46300000e-03 ...,   6.34330000e-02
    7.46300000e-03   3.35820000e-02]
 ..., 
 [  1.10000000e+01   4.33980000e-02   1.56970000e-02 ...,   5.90950000e-02
    2.40070000e-02   4.24750000e-02]
 [  1.10000000e+01   7.94520000e-02   2.74000000e-03 ...,   6.84930000e-02
    1.64380000e-02   4.65750000e-02]
 [  1.10000000e+01   6.17280000e-02   2.67490000e-02 ...,   6.99590000e-02
    1.85190000e-02   5.14400000e-02]]
[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 1.  1.  1. ...,  1.  1.  1.]
 [ 1.  1.  1. ...,  1.  1.  1.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 1.  1.  1. ...,  1.  1.  1.]
 [ 1.  1.  1. ...,  1.  1.  1.]]
